- 23.1 Dates and Times in Python
    - Python modules `datetime` and thirdy-party `dateutil`:
        - Flexible and easy to use, but inefficient working with large arrays of dates and times.
        - `from datetime import datetime; datetime(year=2021, month=7, day=4)`
        - `from dateutil import parser; date = parser.parse('4th of July, 2021')`
    - NumPy DType `datetime64`: Typed Arrays of Times
        - Efficient, but lacking the convenient funcs.
        - Defined to be 64-bit, `datetime64` has a trade-off between time resolution and max time span.
        - For typical real-life data, `datetime64[ns]` offers a fine time resolution and +-292 years time span.
        - `np.datetime64('2021-07-04')`    # dtype `datetime64[D]`
        - `np.datetime64('2021-07-04 11:59:59.50', 'ns')`    # dtype `datetime64[ns]`
    - Pandas Dates and Times: Easy to Use and Efficient to Store and Compute
        - `date = pd.to_datetime('4th of July, 2021')`    # flexible parsing
        - `date.strftime(%A)`    # output the day of the week
        - `date + pd.to_timedelta(np.arange(14), 'D')`    # vectorized op
- 23.2 Pandas Time Series: Indexing by Time Avails Pandas Time Series Tools
- 23.3 Pandas Data Structures for Time Series 
    - `pd.Timestamp` is replacement for Python's datetime, based on np.datetime64;
    - `pd.Timedelta` is replacement for Python's datedelta, based on np.timedelta64;
    - `pd.Period` is a new type, a time duration with specific starting and ending timestamps.
    - The corresponding index types are: `pd.DatetimeIndex`, `pd.TimedeltaIndex`, and `pd.PeriodIndex`.
    - `dates.to_period('D')`    # convert a DatetimeIndex obj to a PeriodIndex obj
    - `dates - dates[0]`    # create a TimedeltaIndex
- 23.4 Regular Sequences: `pd.date_range()`, `pd.period_range()`, `pd.timedelta_range()`
    - `pd.date_range('2015-07-03', periods=8, freq='H')`    # 8 hours, starting from 2015-07-03 0:00:00; 
        - or, skip freq= for the default 'D'
    - `pd.period_range('2015-07', periods=8, freq='M')`    # 8 months, starting from 2015-07
    - `pd.timedelta_range(0, periods=6, freq='H')`    # 6 hours
- 23.5 Frequencies and Offsets: Fundamental to Pandas Time Series
    - Year to day and hour: Default marked at the period end: Valendar vs Biz;
    - Minute, second, down to nanosecond;
    - Change to be marked at the period beginning with suffix `S`;
    - Change the end or start month of year or quarter with suffix `-YUE`;
    - Change the end or start day of week with suffix `-DAY`;
    - Combine codes for additional frequencies, e.g. `2H30T` for 150 minutes;
    - See Tables 23-2 and 23-3 on p201.
    - e.g. `pd.timedelta_range(0, periods=6, freq='2H30T')`
    - e.g. `pd.date_range('2015-07-01', periods=7, freq=BDay())`    # BDay() imported prior
- 23.6 Resampling, Shifting, and Windowing
    - Resamping with `.resample()` and converting frequencies with `.asfreq()`
        - `sp500.resample('BA').mean().plot(style=':')`   # data aggregation with .resample()
        - `sp500.asfreq('BA').plot(style='--')`    # data selection with .asfreq()
        - `sp500.asfreq('D', method='ffill').plot()`    # upsampling and forward-filling with .asfreq()
    - Time shifts with `.shift()`
        - `roi = 100 * (sp500.shift(-365) - sp500) / sp500`    # ROI after 1 year
    - Rolling windows with `.rolling()`
        - `mean_1yr_rolling = sp500.rolling(365, center=True).mean()`
- 23.7 Example: Visualizing Seattle Bicycle Counts
    - Check index duplicity: `data.index.duplicated().sum()`    # turned out to be 0, no duplication
    - Check index monotonicity:
        - `data.index.is_monotonic_increasing, data.index.is_monotonic_decreasing`    # turned out T, F
    - Check data points per day: `data.index.to_series().dt.date.value_counts().value_counts()`
    - Weekly, monthly, and yearly variations:
        - Plot weekly sum: `data.resample('W').sum().plot()`
        - Daily sum: `daily = data.resample('D').sum()`
        - Plot rolling 30-day sum: `daily.rolling(30, center=True).sum().plot(style=['-', ':', '--'])`
        - Take care of the ruggedness due to hard window cutoff:
            - `daily.rolling(50, center=True, win_type='gaussian').sum(std=10).plot(style=['-', ':', '--'])`
    - Daily variations:
        - Plot data by weekday: `data.groupby(data.index.dayofweek).mean().plot(style=['-', ':', '--'])`
        - Plot data by hour:
            - `data.groupby(data.index.time).mean().plot(xticks=4 * 60 * 60 * np.arange(6), style=['-', ':', '--'])`
        - Plot weekday vs weekend:
            - `weekend = np.where(data.index.weekday < 5, 'Weekday', 'Weekend')`
            - `by_time2 = data.groupby([weekend, data.index.time]).mean()`
            - `fig, axes = plt.subplots(1, 2, figsize=(14, 5))`
            - `by_time2.loc['Weekday'].plot(ax=axes[0], title='Weekdays', xticks=hourly_ticks, style=['-', ':', '--'])`
            - `by_time2.loc['Weekend'].plot(ax=axes[1], title='Weekends', xticks=hourly_ticks, style=['-', ':', '--'])`